In [1]:
# Pythonの基本ライブラリ
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# ファイル操作
import os

# Jupyter上にHTMLを表示する
from IPython.display import HTML

# 自作モジュールのimport
import util.create_wiki_data as create_wiki_data
import util.annotate_another_data as annotate_another_data
import util.annotate_another_data_auto as annotate_another_data_auto
import util.filter_data as filter_data

In [2]:
# wikiにある情報を取得
streams = create_wiki_data.create_streams()
songs = create_wiki_data.create_songs()

In [2]:
# 既に作成済みのデータを取得
base_df_path = 'data/streams.csv'
base_df = pd.read_csv(base_df_path, header=0)

In [3]:
# 念の為、バックアップを残しておく
base_df_backup_path = 'data/streams_.csv'
base_df.to_csv(base_df_backup_path, index=False)

In [5]:
# バックアップから復元できるようにもしておく
reboot = False
if reboot:
    override_path = 'data/streams_.csv'
    base_df_backup = pd.read_csv(override_path, header=0)
    base_df_backup.to_csv(base_df_path, index=False)

In [6]:
# 追加するデータのデータベースを作成
add_streams_path = 'data/add_streams.csv'
if os.path.exists(add_streams_path):
    add_streams = pd.read_csv(add_streams_path, header=0)
else:
    add_streams = streams.tail(5)
    # Youtube API経由で日付けでフィルタリングも可能
    # date = '2022-07-01' # dateはYYYY-mm-ddの形式で入力
    # add_streams = annotate_another_data.filter_add_df(streams, date)
    add_streams['check'] = np.zeros(add_streams.shape[0], dtype=bool)
    add_streams.to_csv(add_streams_path, index=False)

add_songs_path = 'data/add_songs.csv'
if os.path.exists(add_songs_path):
    add_songs = pd.read_csv(add_songs_path, header=0)
else:
    add_songs = songs
    add_songs['check'] = np.zeros(add_songs.shape[0], dtype=bool)
    add_songs.to_csv(add_songs_path, index=False)

In [ ]:
# wikiから取得したstreamsの差分にアノテーションする
annotate_another_data.annotate_another_data_streams(add_streams_path, base_df_path)

In [ ]:
# wikiから取得したsongsの差分にアノテーションする
annotate_another_data.annotate_another_data_songs(add_songs_path, base_df_path)

In [10]:
# 手動で好きな動画にアノテーションする
# とりあえず5~8月のおすすめを探す旅に出る

list_name = '8月のおすすめ'
annotate_another_data.annotate_another_data_manual(base_df_path, list_name)

動画のURLを入力


 https://www.youtube.com/watch?v=VsYtOSHzXHA


曲名を入力


 ココロオドル


開始時刻を入力


...

 獅白ぼたん,戌神ころね,猫又おかゆ


stream,live3d,acappellaを01で入力


 110


動画のURLを入力


KeyboardInterrupt: Interrupted by user

In [11]:
# 追加する動画のサムネ一覧を表示
# dateが0のもののみ表示しています
HTML(annotate_another_data_auto.output_html_diff_video(base_df_path))

In [12]:
# 必要そうな追加情報を自動付与する
annotate_another_data_auto.annotate_another_data_auto(base_df_path)

In [13]:
# 不適当なデータを排除する

# 30秒以下の動画は削除
filter_data.filter_duration(base_df_path)

# 今後はホワイトリストに入ったchannelID以外外すなどが考えられる

In [14]:
# 暫定的にjson形式で保存する
base_df_for_save = pd.read_csv(base_df_path, header=0)

output_path = 'data/suise_song.csv'
base_df_for_save.to_csv(output_path, index=False)
output_path = 'data/suisei_songs.json'
base_df_for_save.to_json(output_path, orient='records')

## 以下では必要に応じた修正を実施する

In [31]:
# ミスって削除してしまった分を回収
tmp = pd.read_csv('data/tmp.csv', header=0)
identifier_base = 684
data = []
for i, row in tmp.iterrows():            
    identifier = identifier_base + i
    song_name = row['SongNameRemake']
    artist_name = row['ArtistNameRemake']
    start_time = row['StartTimeRemake']
    end_time = row['EndTimeRemake']
    members = row['Members']
    video_id = row['VideoID']
    date = row['Date']
    collaboration = row['Collaboration']
    acappella = row['Acappella']
    live3d = row['3DLive']
    suisei = True
    stream = True
    listtype = 'None'
    check = True

    data.append([identifier, song_name, artist_name, start_time, end_time,
                 members, video_id, date, collaboration, acappella,
                 live3d, suisei, stream, listtype, check])
    
df_tmp = pd.DataFrame(data=np.array(data), 
                      columns=['id', 'songname', 'artistname', 'starttime', 'endtime', 
                               'members', 'videoid', 'date', 'collaboration', 'acappella', 
                               'live3d', 'suisei', 'stream', 'listtype', 'check'])

base_df = pd.concat([base_df, df_tmp])
base_df.to_csv('data/streams.csv', index=False)

In [38]:
# 名称の表記揺れを削除
base_df_path = 'data/streams.csv'
base_df = pd.read_csv(base_df_path, header=0)

song_names = np.array(base_df['songname'])
artist_names = np.array(base_df['artistname'])

unique_song_names = np.unique(song_names)
# print(unique_song_names)

unique_artist_names = np.unique(artist_names)
# print(unique_artist_names)

In [12]:
# Quota上限オーバーでタイムスタンプ取得に失敗したデータをリセット
base_df_ = pd.read_csv('data/streams.csv', header=0)
dates = np.array(base_df_['date'])
dates[742:] = 0
base_df_['date'] = dates
base_df_.to_csv('data/streams.csv', index=False)